In [4]:
import numpy as np
import ipywidgets as widgets
from ipyleaflet import Map, Marker, AwesomeIcon, Icon
from IPython.display import display
from glasstone.overpressure import brode_overpressure
from Thermal import Thermal_Fluence
from Underwater_OP import UOP
from glasstone.radiation import glasstone_summary
from glasstone.utilities import convert_units, ValueOutsideGraphError

# Add checkbox for underwater vs free air
Scenario_Selector = widgets.Checkbox(
    description='Check if it is an underwater detonation',
    disabled=False,
    indent=False
)
display(Scenario_Selector)

Yield=None

def store_value(button):
    global Yield
    Yield=yield_input.value
    Yield=int(Yield)
    yield_text.value=f"Input Yield={Yield} kt"

def store_value2(button):
    global Z
    Z = HOB_input.value + np.random.uniform(0,25)
    Z=int(Z)
    HOB_text.value=f"Input Height Burst={Z} m"


# Function to calculate distance between two coordinates
def calculate_distance(coord1, coord2):
    x1, y1 = coord1
    x2, y2 = coord2
    distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2) * 111320 # Conversion from degrees to m
    return distance

# Function to add a new ship icon
def add_ship(icon_type, ship_color, ship_type, initial_location):
    while True:
        # Generate random location within the map boundaries
        random_location = [
            main_icon.location[0] + np.random.uniform(-0.01, 0.01),
            main_icon.location[1] + np.random.uniform(-0.01, 0.01)
        ]

        # Check distance from existing icons
        too_close = False
        for icon in secondary_icons:
            distance = calculate_distance(icon.location, random_location)
            if distance < 100:
                too_close = True
                break

        if not too_close:
            break

    new_icon = Marker(
        location=random_location,
        draggable=True,
        icon=Icon(icon_url=icon_type, icon_size=[50, 50],marker_color=ship_color, icon_color='white')
    )
    secondary_icons.append(new_icon)
    secondary_ship_types[new_icon] = ship_type
    m.add_layer(new_icon)

# Function to handle main icon movement
def on_main_icon_move(event, main_icon):
    main_icon.location = event['new']

# Function to finalize ship location and calculate distances
def finalize_ship_location(button):
    global distances
    global distances1
    # Move main icon to average position of 100 random movements within 50 meters of the original position
    main_icon_location = main_icon.location
    for _ in range(100):
        random_location = [
            main_icon_location[0] + np.random.uniform(-0.0001, 0.0001),
            main_icon_location[1] + np.random.uniform(-0.0001, 0.0001)
        ]
        distance = calculate_distance(main_icon_location, random_location)

        if distance <= 50:
            main_icon_location = random_location

    main_icon.location = main_icon_location

    # Calculate distances from main icon to all secondary icons
    distances = []
    distances1 = []
    for icon in secondary_icons:
        distance = calculate_distance(main_icon.location, icon.location)
        distances.append(distance)
        distances1.append((secondary_ship_types[icon], distance))

    distances=np.array(distances)
    #Display distances in the text widget1
    distances_text.value = "\n".join(f"{ship}: {distance:.2f} meters" for ship, distance in distances1)

# Function to handle map change
def on_map_change(change):
    if change['name'] == 'center':
        main_icon.location = change['new']

def calculate(button):
    global OP
    OP=[]
    OP1 = []
    if Scenario_Selector.value==False:
        for icon in secondary_icons:
            distance = calculate_distance(main_icon.location, icon.location)
            OP=brode_overpressure(Yield,distance,Z)
            OP=convert_units(OP,'kg/cm^2' ,'psi')
            OP1.append((secondary_ship_types[icon], OP))

        #distances=np.array(distances)
        OP_text.value = "\n".join(f"{ship}: {OP:.2f} psi" for ship, OP in OP1)
        
    elif ((Scenario_Selector.value==True) and (map_selection_dropdown.value == 'Pearl Harbor')):
        for icon in secondary_icons:
            distance = calculate_distance(main_icon.location, icon.location)
            D_w = 14
            OP=UOP(Yield,distance,Z,D_w)
            OP1.append((secondary_ship_types[icon], OP))
             #distances=np.array(distances)
        OP_text.value = "\n".join(f"{ship}: {OP:.2f} psi" for ship, OP in OP1)
            
    elif ((Scenario_Selector.value==True) and (map_selection_dropdown.value == 'Norfolk Naval Station, Virginia')):
        for icon in secondary_icons:
            distance = calculate_distance(main_icon.location, icon.location)
            OP=2
            OP1.append((secondary_ship_types[icon], OP))
             #distances=np.array(distances)
        OP_text.value = "\n".join(f"{ship}: {OP:.2f} psi" for ship, OP in OP1)
            
    elif ((Scenario_Selector.value==True) and (map_selection_dropdown.value == 'Middle of the Atlantic Ocean')):
        for icon in secondary_icons:
            distance = calculate_distance(main_icon.location, icon.location)
            OP=3
            OP1.append((secondary_ship_types[icon], OP))
             #distances=np.array(distances)
        OP_text.value = "\n".join(f"{ship}: {OP:.2f} psi" for ship, OP in OP1)
            
    elif ((Scenario_Selector.value==True) or (map_selection_dropdown.value == 'Middle of the Pacific Ocean')):
        for icon in secondary_icons:
            distance = calculate_distance(main_icon.location, icon.location)
            OP=4
            OP1.append((secondary_ship_types[icon], OP))

        #distances=np.array(distances)
        OP_text.value = "\n".join(f"{ship}: {OP:.2f} psi" for ship, OP in OP1)

def findf(Y,Z):
    global f
    zb=convert_units(Z,'m','kilofeet')
    zb1=zb*1000
    if Y<=1:
        if zb1<= 20:
            f=.19

        elif zb1<= 40 and zb1>20:
            f=.21

        elif zb1<= 70 and zb1>40:
            f=.23

        elif zb1<= 100 and zb1>70:
            f=.26

        elif zb1<= 200 and zb1>100:
            f=.35

        elif zb1>= 200:
            f=.35
    elif Y>1 and Y<=10:
        if zb1<= 20:
            f=.18

        elif zb1<= 40 and zb1>20:
            f=.19

        elif zb1<= 70 and zb1>40:
            f=.21

        elif zb1<= 100 and zb1>70:
            f=.22

        elif zb1<= 200 and zb1>100:
            f=.25

        elif zb1<= 400 and zb1>200:
            f=.33

        elif zb1>= 400:
            f=.35

    elif Y>10:
        if zb1<= 40:
            f=.18

        elif zb1<= 70 and zb1>40:
            f=.19

        elif zb1<= 100 and zb1>70:
            f=.20

        elif zb1<= 200 and zb1>100:
            f=.21

        elif zb1<= 400 and zb1>200:
            f=.25

        elif zb1<= 700 and zb1>400:
            f=.28

        elif zb1<= 1000 and zb1>700:
            f=.34 

        elif zb1>= 1000:
            f=.35

    return f


def calculate2(button):
    global Thermal
    global Thermal1
    Thermal = []
    Thermal1=[]
    f=findf(Yield,Z)
    if Scenario_Selector.value==False:
        for icon in secondary_icons:
            distance = calculate_distance(main_icon.location, icon.location)
            Thermal=Thermal_Fluence(Yield,distance,Z,f)
            Thermal1.append((secondary_ship_types[icon], Thermal))

        Therm_text.value = "\n".join(f"{ship}: {Thermal:.2f} cal/cm^2" for ship, Thermal in Thermal1)
    else:
        for icon in secondary_icons:
            distance = calculate_distance(main_icon.location, icon.location)
            Thermal=0
            Thermal1.append((secondary_ship_types[icon], Thermal))

        Therm_text.value = "\n".join(f"{ship}: {Thermal:.2f} cal/cm^2" for ship, Thermal in Thermal1)

def calculate3(button):
    global Radiation
    global Rad1
    Radiation = []
    Rad1=[]
    for icon in secondary_icons:
        distance = calculate_distance(main_icon.location, icon.location)
        sr = (distance**2+Z**2)**.5
        if sr<500:
            sr = 500
        if convert_units(Z,'m','kilofeet')*1000>300:
            s=False
        elif convert_units(Z,'m','kilofeet')*1000<=300:
            s=True
        Radiation=glasstone_summary(Yield,sr,Z,offense=False,surface=s)*.877
        Rad1.append((secondary_ship_types[icon],Radiation))

    #distances=np.array(distances)
    Rad_text.value = "\n".join(f"{ship}: {Radiation:.2f} rads" for ship, Radiation in Rad1)

# Define initial locations for different maps
initial_locations = {
    "Pearl Harbor": [21.3445, -157.9746],
    "Norfolk Naval Station, Virginia": [36.9375, -76.3186],
    "Middle of the Atlantic Ocean": [27.4076, -40.3018],
    "Middle of the Pacific Ocean": [27.3897, 166.5527]
}

# Create map selection dropdown widget
map_selection_dropdown = widgets.Dropdown(
    options=list(initial_locations.keys()),
    description='Select Map:',
    disabled=False
)

# Create main map
m = Map(center=initial_locations[map_selection_dropdown.value], zoom=15, layout=dict(height='600px'))

# Main icon
main_icon = Marker(
    location=initial_locations[map_selection_dropdown.value],
    draggable=True,
    icon=AwesomeIcon(name='circle', marker_color='red', icon_color='white')
)

# Initialize secondary icons list and ship types dictionary
secondary_icons = []
secondary_ship_types = {}

# Event handler for main icon movement
main_icon.observe(lambda event: on_main_icon_move(event, main_icon), names='location')

# Event handler for map change
m.observe(on_map_change)

# Event handler for map selection change
def on_map_selection_change(change):
    m.center = initial_locations[change['new']]
    main_icon.location = initial_locations[change['new']]

# Bind event handler to map selection dropdown
map_selection_dropdown.observe(on_map_selection_change, names='value')

# Add main icon to the map
m.add_layer(main_icon)

# Display the map selection dropdown
display(map_selection_dropdown)

# Display for the yield input
Yield_text=widgets.Label(value='Yield Input (kt):')
display(Yield_text)
yield_input = widgets.FloatText()
yieldbutton = widgets.Button(description="Enter")
yieldbutton.on_click(store_value)

# Display the KT input
display(yield_input)
display(yieldbutton)

# Text widget to display yield
yield_text = widgets.Textarea(layout={'height': '50px', 'width': '200px'})
display(yield_text)

# Display for HOB input
HOB_text=widgets.Label(value='Height/Depth of Burst Input (m):')
display(HOB_text)
HOB_input = widgets.FloatText()
HOBbutton = widgets.Button(description="Enter")
HOBbutton.on_click(store_value2)

# Display the HOB input
display(HOB_input)
display(HOBbutton)

# Text widget to display HOB
HOB_text = widgets.Textarea(layout={'height': '50px', 'width': '200px'})
display(HOB_text)

# Display the map
display(m)

# Event handler to add a carrier icon
def add_carrier(button):
    add_ship('https://upload.wikimedia.org/wikipedia/commons/6/6e/NATO_Map_Symbol_-_Friendly_Sea_-_Aircraft_Carrier.svg','blue', 'Carrier', m.center)

# Event handler to add a cruiser icon
def add_cruiser(button):
    add_ship('green', 'Cruiser', m.center)

# Event handler to add a destroyer icon
def add_destroyer(button):
    add_ship('black', 'Destroyer', m.center)

# Event handler to add a supply ship icon
def add_supply_ship(button):
    add_ship('white', 'Supply Ship', m.center)


def clear_ships(button):
    for icon in secondary_icons:
        m.remove_layer(icon)
    secondary_icons.clear()

# Add clear button
clear_button = widgets.Button(description="Clear Ships")
clear_button.on_click(clear_ships)

# Add clear button to the display
display(clear_button)

# Add secondary icon buttons
add_carrier_button = widgets.Button(description="Add Carrier")
add_carrier_button.on_click(add_carrier)
add_cruiser_button = widgets.Button(description="Add Cruiser")
add_cruiser_button.on_click(add_cruiser)
add_destroyer_button = widgets.Button(description="Add Destroyer")
add_destroyer_button.on_click(add_destroyer)
add_supply_ship_button = widgets.Button(description="Add Supply Ship")
add_supply_ship_button.on_click(add_supply_ship)

# Display buttons
ship_buttons = widgets.HBox([add_carrier_button, add_cruiser_button, add_destroyer_button, add_supply_ship_button])
display(ship_buttons)
# Display finalize ship location button
finalize_button = widgets.Button(description="Finalize Ship Location")
finalize_button.on_click(finalize_ship_location)
display(finalize_button)

# Text widget to display distances
distances_text = widgets.Textarea(layout={'height': '100px', 'width': '600px'})
display(distances_text)

#OVERPRESSURE BUTTON
OP_text=widgets.Label(value='Peak Overpressure Calculation')
display(OP_text)
OPbutton = widgets.Button(description="Calculate")
OPbutton.on_click(calculate)

# Display the OP output
display(OPbutton)
OP_text = widgets.Textarea(layout={'height': '100px', 'width': '600px'})
display(OP_text)

#THERMAL BUTTON
Therm_text=widgets.Label(value='Thermal Calculation')
display(Therm_text)
Thermbutton = widgets.Button(description="Calculate")
Thermbutton.on_click(calculate2)

# Display the Thermal Output
display(Thermbutton)
Therm_text = widgets.Textarea(layout={'height': '100px', 'width': '600px'})
display(Therm_text)

#Radiation BUTTON
Rad_text=widgets.Label(value='Radiation Calculation')
display(Rad_text)
Radbutton = widgets.Button(description="Calculate")
Radbutton.on_click(calculate3)

# Display the Radiation Output
display(Radbutton)
Rad_text = widgets.Textarea(layout={'height': '100px', 'width': '600px'})
display(Rad_text)

Checkbox(value=False, description='Check if it is an underwater detonation', indent=False)

Dropdown(description='Select Map:', options=('Pearl Harbor', 'Norfolk Naval Station, Virginia', 'Middle of the…

Label(value='Yield Input (kt):')

FloatText(value=0.0)

Button(description='Enter', style=ButtonStyle())

...

Button(description='Calculate', style=ButtonStyle())

Textarea(value='', layout=Layout(height='100px', width='600px'))

Label(value='Radiation Calculation')

Button(description='Calculate', style=ButtonStyle())

Textarea(value='', layout=Layout(height='100px', width='600px'))

In [9]:
map_selection_dropdown.value

'Pearl Harbor'